In [1]:
pip install pymediainfo --trusted-host pypi.org --trusted-host files.pythonhosted.org

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 8.6 MB/s eta 0:00:00


In [2]:
pip install progressbar --trusted-host pypi.org --trusted-host files.pythonhosted.org

  Preparing metadata (setup.py) ... done
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12065 sha256=afb15fab06ae6d0187cdbd51004bcc594c8a18d9f65c19d2ab7e2578b211d653
  Stored in directory: /root/.cache/pip/wheels/a5/4d/c7/f3cf0f75c746c219090060131fe00f1523cc2c5484991f4030
Successfully built progressbar


In [ ]:
import re
import os
import json
import subprocess
import pandas as pd
import numpy as np
from datetime import datetime
from pymediainfo import MediaInfo
import progressbar

Seleccionar almacenamiento

In [4]:
#Seleccionar Google Drive
from google.colab import drive
drive.mount('/content/drive')
source = r'/content/drive/Shareddrives/Proyecto TM' #con drive

Mounted at /content/drive


In [ ]:
#Seleefionar Local
source =  r'G:/Unidades compartidas/Proyecto TM' #local

In [5]:
#Rutas
src_dataset = source + r'/Notebooks/datasets'
src_videos = source + r'/Videos TM'

In [9]:
#Validar ruta de los videos del dataset
def validar_ruta(ruta):
    if os.path.exists(ruta):
        content = os.listdir(src_videos)
        if(len(content) != 0):
            print(f"Hay {len(content)} videos en la ruta especificada")
            return True
        else:
            print("La ruta no contiene archivos")
            return False
    else:
        print("La ruta no es válida")
        return False


In [10]:
validar_ruta(src_videos)

Hay 1800 videos en la ruta especificada


True

In [11]:

def extraer_fecha_hora(nombre_archivo):
    # Patrón para buscar YYYYMMDDHHMMSS (14 dígitos entre 'Ch#_ 'y '-XX.mkv')
    patron = r'Ch\d_(\d{14})-\d{2}\.mkv'

    # Buscar el patrón
    coincidencia = re.match(patron, nombre_archivo)
    if coincidencia:
        fecha_str = coincidencia.group(1)
        try:
            # Convertir a formato datetime y luego a string legible
            fecha = datetime.strptime(fecha_str, '%Y%m%d%H%M%S')
            return fecha.strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            return np.nan
    return np.nan


In [12]:
def obtener_metadatos(ruta_video):
  try:
      media_info = MediaInfo.parse(ruta_video)
      metadatos = {}
      for track in media_info.tracks:
          if track.track_type == "General":
            for key, value in track.to_data().items():
              if value and isinstance(value, str):
                  metadatos[key.lower()] = value
              elif key == 'duration':
                metadatos['duracion'] = float(value / 1000)
            break
          else:
            raise Exception("No se encontró el track General en el archivo")
      return metadatos

  except Exception as e:
      print(f"Error al analizar el archivo: {e}")
      return None

In [ ]:
#Extraer metadatos de los videos
def extraer_metadatos(ruta_video):
    try:

        data = obtener_metadatos(ruta_video)

        duracion = data.get('duracion', 0)
        evasion = data.get('evasion', 'N/A')
        categoria = data.get('categoria', 'N/A')
        cantidad = int(data.get('cantidad', 0))
        orientacion = data.get('orientacion', 'N/A')
        complejo = data.get('complejo', 'N/A')
        estacion = data.get('estacion', 'N/A')

        return {
            'nombre': os.path.basename(ruta_video),
            'evasion': evasion,
            'categoria': categoria,
            'cantidad': cantidad,
            'complejo': complejo,
            'duracion': duracion,
            'orientacion': orientacion,
            'estacion': estacion,
            'fecha': extraer_fecha_hora(os.path.basename(ruta_video))
        }
    except Exception as e:
        print(f"Error en {ruta_video}: {e}")
        return None

-----------------------------------------------


In [14]:
ruta_valida = validar_ruta(src_videos)

if(ruta_valida):
    # Listar todos los .mkv
    rutas_videos = [os.path.join(src_videos, f) for f in os.listdir(src_videos) if f.endswith('.mkv')]

    bar = progressbar.ProgressBar(maxval=len(rutas_videos), widgets=[
        ' [', progressbar.Percentage(), '] ',
        progressbar.Bar(), ' ', progressbar.ETA()
    ])
    bar.start()

    # Extraer datos
    datos = []
    for i, ruta_video in enumerate(rutas_videos):
        bar.update(i + 1)
        info = extraer_metadatos(ruta_video)
        if info:
            datos.append(info)

    # Crear DataFrame
    df = pd.DataFrame(datos)

    bar.finish()


print('finalizado')


Hay 1800 videos en la ruta especificada


finalizado


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   nombre       1800 non-null   object 
 1   evasion      1800 non-null   object 
 2   categoria    1800 non-null   object 
 3   cantidad     1800 non-null   int64  
 4   complejo     1800 non-null   object 
 5   duracion     1800 non-null   float64
 6   orientacion  1800 non-null   object 
 7   estacion     1800 non-null   object 
 8   fecha        1300 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 126.7+ KB


In [16]:
df.head(100)

,nombre,evasion,categoria,cantidad,complejo,duracion,orientacion,estacion,fecha
0,Ch2_20181118181010-09.mkv,SI,CASO_1,11,NO,21.0,BACK,NIEVES,2018-11-18 18:10:10
1,Ch2_20181118181010-10.mkv,SI,CASO_1,5,NO,21.0,BACK,NIEVES,2018-11-18 18:10:10
2,Ch2_20181118181010-11.mkv,SI,CASO_1,3,NO,20.0,BACK,NIEVES,2018-11-18 18:10:10
3,Ch2_20181118181010-12.mkv,SI,CASO_1,3,NO,20.0,BACK,NIEVES,2018-11-18 18:10:10
4,Ch2_20181118181010-13.mkv,SI,CASO_1,1,NO,21.0,BACK,NIEVES,2018-11-18 18:10:10
...,...,...,...,...,...,...,...,...,...
95,Ch1_20181019173335-01.mkv,SI,CASO_1,1,NO,21.0,BACK,POLICARPA,2018-10-19 17:33:35
96,Ch1_20181019180935-00.mkv,SI,CASO_1,1,NO,21.0,BACK,POLICARPA,2018-10-19 18:09:35
97,Ch1_20181020112301-00.mkv,SI,CASO_1,1,NO,21.0,BACK,POLICARPA,2018-10-20 11:23:01
98,Ch1_20181022063425-00.mkv,SI,CASO_1,1,NO,20.0,BACK,POLICARPA,2018-10-22 06:34:25


In [17]:
#Exportar dataset
dataset_ruta = src_dataset + r'/dataset_tm_full.csv'
df.to_csv(dataset_ruta, index=False)
print('dataset exportado en ' + dataset_ruta)


dataset exportado en /content/drive/Shareddrives/Proyecto TM/Notebooks/datasets/dataset_tm_full.csv
